# XGBoost

In [93]:
import xgboost as xgb
import os

In [120]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

In [95]:
merge_features = '"{}"'.format(os.path.join(current_folder, '..', 'Features', 'Merge features.ipynb'))

In [96]:
calcular_auc = '"{}"'.format(os.path.join(current_folder, '..', 'Calcular AUC.ipynb'))

In [121]:
%run $merge_features

KeyboardInterrupt: La limpieza ya corrió en este Kernel

In [122]:
set_de_entrenamiento_y_testing = '"{}"'.format(os.path.join(current_folder, '..', 'Set de entrenamiento y testing.ipynb'))

In [123]:
%run $set_de_entrenamiento_y_testing

In [124]:
training = labels_training.merge(df_features, how='inner', on='person')

In [125]:
training.head()

,label,screen_resolution_height,screen_resolution_width,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,event_count
person,,,,,,,,,,,,,,,
0566e9c1,0,568.0,320.0,6.0,3.0,1.0,1.0,15.0,0.0,1.0,0.0,1.0,23.0,17.0,68
6ec7ee77,0,640.0,360.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2
abe7a2fb,0,640.0,360.0,9.0,14.0,1.0,0.0,9.0,0.0,4.0,6.0,0.0,31.0,22.0,96
34728364,0,640.0,360.0,0.0,5.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,24.0,4.0,37
87ed62de,0,864.0,1152.0,5.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,17


In [126]:
columns = list(training.columns)
columns.remove('label')

In [127]:
dtrain = xgb.DMatrix(training.loc[:, columns], label=training['label'])

Hiperparámetros:

In [128]:
param = {'max_depth': 2, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'

In [129]:
num_round = 10
bst = xgb.train(param, dtrain, num_round)

Predecir:

In [130]:
labels_to_predict.head()

""
person
842bc437
993aa8ee
2b0d53b8
4b99307b
e637a01b


In [131]:
labels_to_predict = labels_to_predict.merge(df_features, how='inner', on='person')

In [132]:
prediction = xgb.DMatrix(labels_to_predict.loc[:, columns])

In [133]:
labels_to_predict['label'] = bst.predict(prediction)

Testear la predicción:

In [134]:
%run $calcular_auc

auc score: 0.8456222156222155


In [52]:
import xgboost as xgb
import os.path

In [53]:
%run "Condensación de registros.ipynb"

In [54]:
%run "Predecir y testear con labels conocidos.ipynb"

In [81]:
dataset = 'trocafone_kaggle_test.csv'
labels_to_predict = pd.read_csv(os.path.join('fiuba-trocafone-tp2-final-set', dataset), low_memory=False, index_col='person')

In [83]:
training = labels.merge(df_features, how='inner', on='person')

In [84]:
training.head()

,label,screen_resolution_height,screen_resolution_width,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,event_count
person,,,,,,,,,,,,,,,
0566e9c1,0,568.0,320.0,6.0,3.0,1.0,1.0,15.0,0.0,1.0,0.0,1.0,23.0,17.0,68
6ec7ee77,0,640.0,360.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2
abe7a2fb,0,640.0,360.0,9.0,14.0,1.0,0.0,9.0,0.0,4.0,6.0,0.0,31.0,22.0,96
34728364,0,640.0,360.0,0.0,5.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,24.0,4.0,37
87ed62de,0,864.0,1152.0,5.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,17


In [85]:
columns = list(training.columns)
columns

['label',
 'screen_resolution_height',
 'screen_resolution_width',
 'ad campaign hit',
 'brand listing',
 'checkout',
 'conversion',
 'generic listing',
 'lead',
 'search engine hit',
 'searched products',
 'staticpage',
 'viewed product',
 'visited site',
 'event_count']

In [86]:
columns.remove('label')
columns

['screen_resolution_height',
 'screen_resolution_width',
 'ad campaign hit',
 'brand listing',
 'checkout',
 'conversion',
 'generic listing',
 'lead',
 'search engine hit',
 'searched products',
 'staticpage',
 'viewed product',
 'visited site',
 'event_count']

In [87]:
dtrain = xgb.DMatrix(training.loc[:, columns], label=training['label'])

In [88]:
param = {'max_depth': 2, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'

In [89]:
num_round = 10
bst = xgb.train(param, dtrain, num_round)

In [94]:
labels_to_predict = labels_to_predict.merge(df_features, how='inner', on='person')

In [95]:
dpred = xgb.DMatrix(labels_to_predict.loc[:, columns])

In [96]:
ypred = bst.predict(dpred)

In [100]:
labels_to_predict['label'] = ypred

In [104]:
labels_to_predict.loc[:, ['label']].to_csv(path_or_buf='predictions.csv')